In [1]:
import pandas as pd
import numpy as np

#TO DO:'더치페이 요청에 때한 응답률이 높을수록 더치페이 서비스를 더 많이 사용한다' 통계적 검정
#가정: 요청 응답률 = 더치페이 요청 수신자가 받은 모든 요청건 대비 status가 sent (송금완료) 또는 check (송금 후 요청자 확인완료) 상태인 요청 비율
#본 과제에서 제시하는 가설이 '요청 응답률'라는 변수가 '서비스 사용횟수'라는 변수와 함께 움직이는지, 즉 상관관계가 있는지를 알기 위해 세워진 가설로 해석해서
#그 두 변수간 상관계수(Pearson correlation coefficient 혹은 r: -1이나 1에 가까울수록 상관관계가 확실함)를 계산하는 방식으로 문제를 접근했습니다

In [2]:
data = pd.read_csv('dutchpay_claim_detail.csv')
data.head()

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
0,12918735,4075714,39476d42bd5f268,4,NaN,CLAIM
1,12918734,4075714,a84a2bf8ab324d3,4,4.0,CHECK
2,12829743,4049182,810b763a8f463a9,11750,NaN,CLAIM
3,12829742,4049182,1fc6c9d423a40ec,11750,NaN,CLAIM
4,12829741,4049182,3fc7171d336cf2f,11750,NaN,CLAIM


In [3]:
data.describe()

,claim_detail_id,claim_id,claim_amount,send_amount
count,5.576440e+05,5.576440e+05,5.576440e+05,3.020130e+05
mean,1.334670e+07,4.199503e+06,3.232808e+04,2.756122e+04
std,9.932543e+05,2.851664e+05,2.140968e+05,1.756429e+05
min,1.164020e+07,3.710950e+06,0.000000e+00,0.000000e+00
25%,1.248572e+07,3.952162e+06,6.000000e+03,6.600000e+03
50%,1.333986e+07,4.198393e+06,1.025000e+04,1.066600e+04
75%,1.420613e+07,4.446089e+06,1.862500e+04,1.900000e+04
max,1.507620e+07,4.693812e+06,5.000000e+06,5.000000e+06


In [4]:
data[data['claim_amount']==0].head()

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
45,15042406,4684361,d3044177ef09929,0,0.0,CHECK
46,15042408,4684361,0d4e99769db30d6,0,NaN,CLAIM
47,15042409,4684361,c2b774a45734ad4,0,NaN,CLAIM
48,15042407,4684361,03b3c050f139189,0,NaN,CLAIM
62,15040256,4683754,3bd977e530c3cb0,0,NaN,CLAIM


In [5]:
#0원 송금 요청에 대해 수신자가 응답할 필요가 없다고 판단하여, 요청 응답률 계산에서 제외 처리함
data = data[data['claim_amount']>0]
data.describe()

,claim_detail_id,claim_id,claim_amount,send_amount
count,5.417470e+05,5.417470e+05,5.417470e+05,2.970050e+05
mean,1.334955e+07,4.200324e+06,3.327671e+04,2.800758e+04
std,9.947730e+05,2.856000e+05,2.171427e+05,1.770723e+05
min,1.164020e+07,3.710950e+06,1.000000e+00,1.000000e+00
25%,1.248604e+07,3.952251e+06,6.428000e+03,6.800000e+03
50%,1.334613e+07,4.200146e+06,1.062800e+04,1.087500e+04
75%,1.421174e+07,4.447712e+06,1.900000e+04,1.911200e+04
max,1.507620e+07,4.693812e+06,5.000000e+06,5.000000e+06


In [6]:
#status가 CLAIM이 아닌 요청건들을 '성공'으로 정의
claims_per_receiver = data.groupby('recv_user_id')['claim_id'].nunique()
success_per_receiver = data[data['status']!='CLAIM'].groupby('recv_user_id')['claim_id'].nunique()
receiver_data = pd.merge(claims_per_receiver, success_per_receiver, on='recv_user_id', how='left', suffixes=('_all', '_success'))
receiver_data.head(10)

,claim_id_all,claim_id_success
recv_user_id,,
000109bfc821bda,1,NaN
0001220f2933b14,1,NaN
0001440cbac4d21,1,1.0
00017188c9f4198,1,NaN
0001917bc1a49b5,1,NaN
0001d33a536909b,3,1.0
0001f7e9c43abf0,1,1.0
00020ba234abeaa,1,1.0
000275b5e2e4cf3,1,NaN


In [7]:
#이 '성공' 건들을 요청 받은 유저가 받은 모든 요청건과 비교해서 '응답률'을 계산
receiver_data = receiver_data.fillna(0)
receiver_data['response_rate'] = 100.0*receiver_data['claim_id_success']/receiver_data['claim_id_all']
receiver_data.index.names = ['user_id']
receiver_data.head(10)

,claim_id_all,claim_id_success,response_rate
user_id,,,
000109bfc821bda,1,0.0,0.000000
0001220f2933b14,1,0.0,0.000000
0001440cbac4d21,1,1.0,100.000000
00017188c9f4198,1,0.0,0.000000
0001917bc1a49b5,1,0.0,0.000000
0001d33a536909b,3,1.0,33.333333
0001f7e9c43abf0,1,1.0,100.000000
00020ba234abeaa,1,1.0,100.000000
000275b5e2e4cf3,1,0.0,0.000000


In [8]:
receiver_data.describe()

,claim_id_all,claim_id_success,response_rate
count,235408.000000,235408.000000,235408.000000
mean,2.301307,1.261661,48.466846
std,3.637349,2.730158,48.509322
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,1.000000,1.000000,50.000000
75%,2.000000,1.000000,100.000000
max,156.000000,99.000000,100.000000


In [9]:
#요청 받은 유저들이 더치페이를 요청한 이력이 얼마나 있는지 조사
claimer_data = pd.read_csv('dutchpay_claim.csv')
claimer_data.head()

,claim_id,claim_at,claim_user_id
0,4420721,2020-02-07 15:29:18,5cbd74112c55a0a
1,4420704,2020-02-07 15:26:54,5cbd74112c55a0a
2,4454342,2020-02-10 19:18:31,f077bc4ec8fd0ef
3,4453683,2020-02-10 18:15:11,f077bc4ec8fd0ef
4,4348820,2020-02-01 09:41:30,f077bc4ec8fd0ef


In [10]:
#claims 데이터에서 요청자 id별로 group해서 유저당 요청건수 산출 후 요청 응답률과 합침
claims_per_user = claimer_data.groupby('claim_user_id').claim_id.nunique()
claims_per_user.index.names = ['user_id']
df = pd.merge(receiver_data, claims_per_user, on='user_id', how='left')
df

,claim_id_all,claim_id_success,response_rate,claim_id
user_id,,,,
000109bfc821bda,1,0.0,0.000000,NaN
0001220f2933b14,1,0.0,0.000000,NaN
0001440cbac4d21,1,1.0,100.000000,1.0
00017188c9f4198,1,0.0,0.000000,NaN
0001917bc1a49b5,1,0.0,0.000000,NaN
0001d33a536909b,3,1.0,33.333333,NaN
0001f7e9c43abf0,1,1.0,100.000000,1.0
00020ba234abeaa,1,1.0,100.000000,NaN
000275b5e2e4cf3,1,0.0,0.000000,NaN


In [11]:
df = df.fillna(0)

In [12]:
#응답률과 요청건간의 상관관계 계수(correlation coefficient)를 계산
a = np.array(df['response_rate'])
b = np.array(df['claim_id'])
r = np.corrcoef(a, b)
r
#상관관계 계수가 0.26임으로 매우 약한 상관관계에 속함

array([[1.       , 0.2601404],
       [0.2601404, 1.       ]])